In [127]:
import os
import codecs
import json

businesses_filepath = 'yelp_dataset_challenge_academic_dataset/business.json'

with codecs.open(businesses_filepath, encoding='utf_8') as f:
    first_business_record = f.readline() 

#example of a business summary
print(json.loads(first_business_record))

{'stars': 4.0, 'categories': ['Dentists', 'General Dentistry', 'Health & Medical', 'Oral Surgeons', 'Cosmetic Dentists', 'Orthodontists'], 'state': 'AZ', 'business_id': 'FYWN1wneV18bWNgQjJ2GNg', 'attributes': {'BusinessAcceptsCreditCards': True, 'ByAppointmentOnly': True, 'AcceptsInsurance': True}, 'hours': {'Friday': '7:30-17:00', 'Monday': '7:30-17:00', 'Thursday': '7:30-17:00', 'Wednesday': '7:30-17:00', 'Tuesday': '7:30-17:00'}, 'address': '4855 E Warner Rd, Ste B9', 'name': 'Dental by Design', 'latitude': 33.3306902, 'is_open': 1, 'neighborhood': '', 'postal_code': '85044', 'longitude': -111.9785992, 'city': 'Ahwatukee', 'review_count': 22}


In [128]:
review_json_filepath = 'yelp_dataset_challenge_academic_dataset/review.json'

with codecs.open(review_json_filepath, encoding='utf_8') as f:
    first_review_record = f.readline()
    
#example of a review
print(json.loads(first_review_record))

{'stars': 5, 'useful': 0, 'date': '2016-05-28', 'cool': 0, 'business_id': '0W4lkclzZThpx3V65bVgig', 'review_id': 'v0i_UHJMo_hPBq9bxWvW4w', 'text': "Love the staff, love the meat, love the place. Prepare for a long line around lunch or dinner hours. \n\nThey ask you how you want you meat, lean or something maybe, I can't remember. Just say you don't want it too fatty. \n\nGet a half sour pickle and a hot pepper. Hand cut french fries too.", 'user_id': 'bv2nCi5Qv5vroFiqKGopiw', 'funny': 0}


In [129]:
#our goal is to get a bmuch of business IDs in the restaurant industry
restaurant_ids = set()

# open the businesses file
with codecs.open(businesses_filepath, encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        
        # if this business is not a restaurant, skip to the next one
        if u'Restaurants' not in business[u'categories']:
            continue
            
        # add the restaurant business id to our restaurant_ids set
        restaurant_ids.add(business[u'business_id'])

# turn restaurant_ids into a frozenset, as we don't need to change it anymore
restaurant_ids = frozenset(restaurant_ids)

# print the number of unique restaurant ids in the dataset
print('{:,}'.format(len(restaurant_ids)), u'restaurants in the dataset.')

54,618 restaurants in the dataset.


In [130]:
review_txt_filepath = 'yelp_dataset_challenge_academic_dataset/review_text_all.txt'

In [167]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if True:
    
    review_count = 0

    # create & open a new file in write mode
    with codecs.open(review_txt_filepath, 'w', encoding='utf_8') as review_txt_file:

        # open the existing review json file
        with codecs.open(review_json_filepath, encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
            for review_json in review_json_file:
                review = json.loads(review_json)

                # if this review is not about a restaurant, skip to the next one
                if review[u'business_id'] not in restaurant_ids:
                    continue

                # write the restaurant review as a line in the new file
                # escape newline characters in the original review text
                # making a small change here, need to change it back later
                #review_txt_file.write(review[u'text'].replace('\n', '\\n') + '\n') <-------------------
                if review_count < 5000:
                    review_txt_file.write(review[u'text'].replace('\n', '\\n') + '\n')
                review_count += 1

    print('Text from {:,} restaurant reviews written to the new txt file'.format(review_count))
    
else:
    
    with codecs.open(review_txt_filepath, encoding='utf_8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print(u'Text from {:,} restaurant reviews in the txt file.'.format(review_count + 1))

Text from 3,221,419 restaurant reviews written to the new txt file
Wall time: 2min 31s


In [133]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

In [170]:
# extracting a review for testing
with codecs.open(review_txt_filepath, encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')

print(sample_review)

This is currently my parents new favourite restaurant. 

We come here in the morning for dim sum. They are not the cart pushing type of dim sum, it is order off of the sheet. Dim sum is not bad and not expensive either.

We also frequent the dinner scene. Their set dinner menu is not bad. We typically order a 6 dish menu and it's big enough to feed a family of 9 with leftovers. 

Overall, food is pretty tasty!



In [173]:
%%time
#using NLP (SpaCy English) to parse the review text
parsed_review = nlp(sample_review)

Wall time: 29.6 ms


In [174]:
#the parsed object is actually not all that different. but it actually contains a lot of info
print(type(parsed_review))
print(parsed_review)

<class 'spacy.tokens.doc.Doc'>
This is currently my parents new favourite restaurant. 

We come here in the morning for dim sum. They are not the cart pushing type of dim sum, it is order off of the sheet. Dim sum is not bad and not expensive either.

We also frequent the dinner scene. Their set dinner menu is not bad. We typically order a 6 dish menu and it's big enough to feed a family of 9 with leftovers. 

Overall, food is pretty tasty!



In [175]:
#for example, we can print out the sentences from the parsed object
for num, sentence in enumerate(parsed_review.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

Sentence 1:
This is currently my parents new favourite restaurant. 



Sentence 2:
We come here in the morning for dim sum.

Sentence 3:
They are not the cart pushing type of dim sum, it is order off of the sheet.

Sentence 4:
Dim sum is not bad and not expensive either.



Sentence 5:
We also frequent the dinner scene.

Sentence 6:
Their set dinner menu is not bad.

Sentence 7:
We typically order a 6 dish menu and it's big enough to feed a family of 9 with leftovers. 



Sentence 8:
Overall, food is pretty tasty!




In [176]:
#we can also look at entities
for num, entity in enumerate(parsed_review.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print('')

Entity 1: 6 - CARDINAL

Entity 2: 9 - CARDINAL

Entity 3: 
 - GPE



In [177]:
token_text = [token.orth_ for token in parsed_review] #Verbatim text content 
token_pos = [token.pos_ for token in parsed_review] #part-of-speech.

pd.DataFrame(list(zip(token_text, token_pos)), columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,This,DET
1,is,VERB
2,currently,ADV
3,my,ADJ
4,parents,NOUN
5,new,ADJ
6,favourite,ADJ
7,restaurant,NOUN
8,.,PUNCT
9,\n\n,SPACE


In [178]:
token_lemma = [token.lemma_ for token in parsed_review] # Base form of the token, with no inflectional suffixes.
token_shape = [token.shape_ for token in parsed_review] # Transform of the tokens's string, to show orthographic features. For example, "Xxxx" or "dd".

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)), columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,This,this,Xxxx
1,is,be,xx
2,currently,currently,xxxx
3,my,-PRON-,xx
4,parents,parent,xxxx
5,new,new,xxx
6,favourite,favourite,xxxx
7,restaurant,restaurant,xxxx
8,.,.,.
9,\n\n,\n\n,\n\n


In [179]:
token_entity_type = [token.ent_type_ for token in parsed_review] #Named entity type.
token_entity_iob = [token.ent_iob_ for token in parsed_review] #IOB code of named entity tag. "B" means the token begins an entity, "I" means it is inside an entity, "O" means it is outside an entity, and "" means no entity tag is set.

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
             columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,This,,O
1,is,,O
2,currently,,O
3,my,,O
4,parents,,O
5,new,,O
6,favourite,,O
7,restaurant,,O
8,.,,O
9,\n\n,,O


In [180]:
token_attributes = [(token.orth_, #Verbatim text content
                     token.prob, #Smoothed log probability estimate of token's type
                     token.is_stop, #Is the token part of a "stop list"
                     token.is_punct, #Is the token punctuation
                     token.is_space, #Does the token consist of whitespace characters
                     token.like_num, #Does the token represent a number
                     token.is_oov) #Is the token out-of-vocabulary
                    for token in parsed_review]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: u'Yes' if x else u''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,This,-20.0,,,,,Yes
1,is,-20.0,Yes,,,,Yes
2,currently,-20.0,,,,,Yes
3,my,-20.0,Yes,,,,Yes
4,parents,-20.0,,,,,Yes
5,new,-20.0,,,,,Yes
6,favourite,-20.0,,,,,Yes
7,restaurant,-20.0,,,,,Yes
8,.,-20.0,,Yes,,,Yes
9,\n\n,-20.0,,,Yes,,Yes


## we pretty much used SpaCy as text clean so far. We are going to use gensim to do phrase modeling

1. Segment text of complete reviews into sentences & normalize text
2. First-order phrase modeling → apply first-order phrase model to transform sentences
3. Second-order phrase modeling → apply second-order phrase model to transform sentences
4. Apply text normalization and second-order phrase model to text of complete reviews

We'll use this transformed data as the input for some higher-level modeling approaches in the following sections.

In [181]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [182]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename), batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent if not punct_space(token)])

In [185]:
unigram_sentences_filepath = 'yelp_dataset_challenge_academic_dataset/unigram_sentences_all.txt'

In [186]:
%%time
# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if True:
    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(review_txt_filepath):
            f.write(sentence + '\n')

Wall time: 40.8 s


In [187]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [189]:
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print(' '.join(unigram_sentence))
    print('')

-PRON- be good but after 4 dish already this be just a little too rich

-PRON- come with banana bread that be really good

overall definitely worth the try

just make sure -PRON- bring cash =)

omg come here with some friend for brunch

and wow

-PRON- do have to wait for about 30 min for a table for 4 but -PRON- be later in the day

look at -PRON- menu -PRON- want to try everything

the french toast the strawberry shortcake donut -PRON- be sell out by the time -PRON- get there

but -PRON- wind up only order two thing the classic 2 egg any style house cure bacon or sage and onion sausage crispy griddle red skin potato with sea salt and thyme



In [190]:
bigram_model_filepath = 'yelp_dataset_challenge_academic_dataset/bigram_model_all'

In [194]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if True:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 528 ms


In [195]:
bigram_sentences_filepath = 'yelp_dataset_challenge_academic_dataset/bigram_sentences_all.txt'

In [197]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if True:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 936 ms


In [198]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [199]:
for bigram_sentence in it.islice(bigram_sentences, 230, 240):
    print(' '.join(bigram_sentence))
    print('')

-PRON- be good but after 4 dish already this be just a little too rich

-PRON- come with banana bread that be really good

overall definitely_worth the try

just make_sure -PRON- bring cash =)

omg come_here with some friend for brunch

and wow

-PRON- do have to wait for about 30 min for a table for 4 but -PRON- be later in the day

look at -PRON- menu -PRON- want_to try everything

the french_toast the strawberry shortcake donut -PRON- be sell out by the time -PRON- get there

but -PRON- wind up only order two thing the classic 2 egg any style house cure bacon or sage and onion sausage crispy griddle red skin potato with sea salt and thyme



In [200]:
trigram_model_filepath = 'yelp_dataset_challenge_academic_dataset/trigram_model_all'

In [202]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if True:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 488 ms


In [203]:
trigram_sentences_filepath = 'yelp_dataset_challenge_academic_dataset/trigram_sentences_all.txt'

In [204]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if True:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 946 ms


In [205]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [206]:
for trigram_sentence in it.islice(trigram_sentences, 230, 240):
    print(' '.join(trigram_sentence))
    print('')

-PRON- be good but after 4 dish already this be just a little too rich

-PRON- come_with banana bread that be really good

overall definitely_worth the try

just make_sure -PRON- bring cash =)

omg come_here with some friend for brunch

and wow

-PRON- do have to wait for about 30 min for a table for 4 but -PRON- be later in the day

look_at -PRON- menu -PRON- want_to_try everything

the french_toast the strawberry shortcake donut -PRON- be sell out by the time -PRON- get there

but -PRON- wind up only order two thing the classic 2 egg any style house cure bacon or sage and onion sausage crispy griddle red skin potato with sea salt and thyme



In [207]:
trigram_reviews_filepath = 'yelp_dataset_challenge_academic_dataset/trigram_transformed_reviews_all.txt'

In [250]:
%%time
from spacy.lang.en.stop_words import STOP_WORDS
# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if True:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review if term not in STOP_WORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = ' '.join(trigram_review)
            f.write(trigram_review + '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 42.7 s


In [251]:
print('Original:' + '\n')

for review in it.islice(line_review(review_txt_filepath), 11, 12):
    print(review)

print('----' + '\n')
print('Transformed:' + '\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 11, 12):
        print(review)

Original:

This place is awesome! Definitely authentic!!!

My two favourite dishes are the rice flour rolls and the chicken pho. The rice flour rolls are always fresh whenever I'm there! And the chicken pho is always flavourful!! mmmm....just thinking of it makes me want some!

My boyfriend is Vietnamese and he agrees that this place is authentic and one of the best Vietnamese restaurants he has ever eaten at.

Prices are very reasonable too!

----

Transformed:

this_place awesome definitely authentic -PRON- favourite dish rice flour roll chicken pho rice flour roll fresh -PRON- chicken pho flavourful mmmm think -PRON- -PRON- want -PRON- boyfriend vietnamese -PRON- agree_that this_place authentic good vietnamese restaurant -PRON- have_ever eat_at price reasonable



## we are finally going to topic model with LDA (Latent Dirichlet Allocation)

In [224]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
#import cPickle as pickle #for python 3.x this is not needed
import pickle

In [225]:
trigram_dictionary_filepath = 'yelp_dataset_challenge_academic_dataset/trigram_dict_all.dict'

In [226]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if True:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

Wall time: 74.1 ms


In [227]:
trigram_bow_filepath = 'yelp_dataset_challenge_academic_dataset/trigram_bow_corpus_all.mm'

In [228]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [229]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if True:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

Wall time: 168 ms


In [230]:
lda_model_filepath = 'yelp_dataset_challenge_academic_dataset/lda_model_all'

In [231]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if True:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=50,
                           id2word=trigram_dictionary,
                           workers=3)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

Wall time: 15 s


In [232]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print('{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print('{:20} {:.3f}'.format(term, round(frequency, 3)))

In [233]:
explore_topic(topic_number=0)

term                 frequency

or                   0.043
one                  0.038
as                   0.035
all                  0.026
here                 0.025
out                  0.025
which                0.025
when                 0.023
also                 0.022
if                   0.021
will                 0.021
really               0.020
just                 0.018
do                   0.017
can                  0.016
would                0.016
from                 0.015
say                  0.015
because              0.014
at                   0.014
take                 0.013
other                0.013
again                0.013
more                 0.013
about                0.013


In [236]:
topic_names = {0: 'mexican',
               1: 'menu',
               2: 'thai',
               3: 'steak',
               4: 'donuts & appetizers',
               5: 'specials',
               6: 'soup',
               7: 'wings, sports bar',
               8: 'foreign language',
               9: 'las vegas',
               10: 'chicken',
               11: 'aria buffet',
               12: 'noodles',
               13: 'ambience & seating',
               14: 'sushi',
               15: 'arizona',
               16: 'family',
               17: 'price',
               18: 'sweet',
               19: 'waiting',
               20: 'general',
               21: 'tapas',
               22: 'dirty',
               23: 'customer service',
               24: 'restrooms',
               25: 'chinese',
               26: 'gluten free',
               27: 'pizza',
               28: 'seafood',
               29: 'amazing',
               30: 'eat, like, know, want',
               31: 'bars',
               32: 'breakfast',
               33: 'location & time',
               34: 'italian',
               35: 'barbecue',
               36: 'arizona',
               37: 'indian',
               38: 'latin & cajun',
               39: 'burger & fries',
               40: 'vegetarian',
               41: 'lunch buffet',
               42: 'customer service',
               43: 'taco, ice cream',
               44: 'high cuisine',
               45: 'healthy',
               46: 'salad & sandwich',
               47: 'greek',
               48: 'poor experience',
               49: 'wine & dine'}

In [238]:
topic_names_filepath = 'yelp_dataset_challenge_academic_dataset/topic_names.pkl'

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

In [239]:
LDAvis_data_filepath = 'yelp_dataset_challenge_academic_dataset/ldavis_prepared'

In [242]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if True:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus, trigram_dictionary)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

Wall time: 38.9 s


In [248]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [249]:
pyLDAvis.display(LDAvis_prepared)

In [ ]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(review_txt_filepath), review_number, review_number+1))[0]

In [ ]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review if term not in STOP_WORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda (topic_number, freq): -freq)
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print '{:25} {}'.format(topic_names[topic_number],
                                round(freq, 3))

In [ ]:
sample_review = get_sample_review(50)
print(sample_review)

In [ ]:
lda_description(sample_review)

In [ ]:
sample_review = get_sample_review(100)
print(sample_review)

In [ ]:
lda_description(sample_review)

In [ ]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = 'yelp_dataset_challenge_academic_dataset/word2vec_model_all'

In [ ]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 0 == 1:

    # initiate the model and perform the first epoch of training
    food2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=4)
    
    food2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):

        food2vec.train(trigram_sentences)
        food2vec.save(word2vec_filepath)
        
# load the finished model from disk
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print(u'{} training epochs so far.'.format(food2vec.train_count))

In [ ]:
print(u'{:,} terms in the food2vec vocabulary.'.format(len(food2vec.vocab)))

In [ ]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in food2vec.vocab.iteritems()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda (term, index, count): -count)

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(food2vec.syn0norm[term_indices, :],
                            index=ordered_terms)

word_vectors

In [ ]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):

        print(u'{:20} {}'.format(word, round(similarity, 3)))

In [ ]:
get_related_terms(u'burger_king')

In [ ]:
get_related_terms(u'happy_hour')

In [ ]:
get_related_terms(u'pasta', topn=20)

In [ ]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = food2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

In [ ]:
word_algebra(add=[u'breakfast', u'lunch'])

In [ ]:
word_algebra(add=[u'lunch', u'night'], subtract=[u'day'])

In [ ]:
word_algebra(add=[u'taco', u'chinese'], subtract=[u'mexican'])

In [ ]:
word_algebra(add=[u'bun', u'mexican'], subtract=[u'american'])

In [ ]:
word_algebra(add=[u'filet_mignon', u'seafood'], subtract=[u'beef'])

In [ ]:
word_algebra(add=[u'coffee', u'snack'], subtract=[u'drink'])

In [ ]:
word_algebra(add=[u'burger_king', u'fine_dining'])

In [ ]:
word_algebra(add=[u"denny_'s", u'fine_dining'])

In [ ]:
word_algebra(add=[u"applebee_'s", u'italian'])

In [ ]:
word_algebra(add=[u"applebee_'s", u'pancakes'])

In [ ]:
word_algebra(add=[u"applebee_'s", u'pizza'])

In [ ]:
word_algebra(add=[u'wine', u'barley'], subtract=[u'grapes'])

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne_input = word_vectors.drop(spacy.en.STOPWORDS, errors=u'ignore')
tsne_input = tsne_input.head(5000)

In [ ]:
tsne_input.head()

In [ ]:
tsne_filepath = 'yelp_dataset_challenge_academic_dataset/tsne_model'

tsne_vectors_filepath = 'yelp_dataset_challenge_academic_dataset/tsne_vectors.npy'

In [ ]:
%%time

if 0 == 1:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input.values)
    
    with open(tsne_filepath, 'w') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath) as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])

In [ ]:
tsne_vectors.head()

In [ ]:
tsne_vectors[u'word'] = tsne_vectors.index

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

In [ ]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, resize, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@word') )

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                 color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value(u'16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);